In [1]:
#!pip install -r requirements.txt
import pandas as pd
import unidecode
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import math
from openpyxl import load_workbook

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmart130\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Number of original, valid and colombian tweets

In [2]:
tweets = pd.read_csv('./../data/texts/all_texts.csv')
print('Initial number of tweets:',len(tweets))

tweets = pd.read_csv('./../data/texts/colombian_tweets.csv')
print('Number of tweets from Colombia:',len(tweets))

tweets = pd.read_csv('./../data/texts/colombian_valid_tweets.csv')
print('Number of VALID tweets from Colombia:',len(tweets))

Initial number of tweets: 7406219
Number of tweets from Colombia: 2006528
Number of VALID tweets from Colombia: 1664903


In [17]:
tweets = pd.read_csv('./../data/texts/texts_from_pickles.csv')

In [19]:
#Counting number of tweets from 2015-1 to 2021-3
# Convert to datetime and filtering by date (only 2015-1 - 2021-3)
dates = tweets['date']
dates = pd.to_datetime(pd.Series(dates))
dates = dates[(dates >= '2014-1-1') & (dates <= '2021-3-31')]
print('Number of tweets from 2015-1 to 2021-3:',len(dates))

Number of tweets from 2015-1 to 2021-3: 3996268


In [23]:
tweets = pd.read_csv('./../data/texts/colombian_valid_tweets.csv')

## Word Frequency

In [3]:
words = []
for text in tweets['text'].astype(str):
    words_list = unidecode.unidecode(text).lower().split() #lowercase version of the words were used
    words_list = [word.translate(str.maketrans('', '', string.punctuation)) for word in words_list]
    words+= words_list  #remove punctuations from the words

In [4]:
#remove stopwords
stop_words = set(stopwords.words('spanish')) 
words = [w for w in words if w not in stop_words] 

#remove meaningless words
meaningless_words = ['d', '5', 'si', 'q', '2018', '', ' ']
words = [w for w in words if w not in meaningless_words] 

In [5]:
#remove unwanted words###
unwanted_words = []
words = [w for w in words if not w.lower() in unwanted_words] 

In [6]:
serie_words = pd.Series(words)

In [7]:
export = serie_words.value_counts().reset_index()[0:100] #only exporting the 100 most frequent words
export.rename(columns={'index':'word',0:'weight'}, inplace=True)
export[['word','weight']].to_csv('words_frequencies.csv',index = False)

## Frequency of specific words

In [2]:
tweets = pd.read_csv('./../data/texts/colombian_valid_tweets.csv')
word = 'hospital'
tweets['word_frequency'] = [min(text.count(word),1) for text in tweets['text'].astype(str).str.lower()] #Remove .lower() if case sentitivity is required

In [3]:
# by year
tweets_count_word_year = tweets.groupby([tweets['date'].map(lambda x: x[0:4])]).count()
# by month
tweets_count_word_month = tweets.groupby([tweets['date'].map(lambda x: x[0:7])]).count()

In [5]:
with pd.ExcelWriter('./../analysis/words_frequency/'+word+'_word_frequency.xlsx') as writer:
    tweets_count_word_year['word_frequency'].to_excel(writer, sheet_name= word+' frequency - yearly')
    tweets_count_word_month['word_frequency'].to_excel(writer, sheet_name= word+' frequency - month')

## Frequency of specific words - 2

In [6]:
tweets = pd.read_csv('./../data/texts/colombian_valid_tweets.csv')
words = ['venezolanos', 'migrantes', 'veneco', 'venecos', 'veneca', 'venecas', 'migracion', 'venezuela', 'colombia', 'gobierno']

#without retweets
tweets['text'] = tweets['text'].astype(str).str.lower()
tweets = tweets.drop_duplicates(subset = 'text', inplace=False)

In [8]:
# export_path = './words_frequency/word_frequencies_specifics_words.xlsx'
export_path = './words_frequency/word_frequencies_specifics_words_nort.xlsx'
words_year = pd.DataFrame()
words_month = pd.DataFrame()
book = load_workbook(export_path)

    
for word in words: 
    tweets['word_frequency'] = [min(text.count(word),1) for text in tweets['text'].astype(str).str.lower()] #Remove .lower() if case sentitivity is required
    # by year
    tweets_count_word_year = tweets.groupby([tweets['date'].map(lambda x: x[0:4])]).sum()
    words_year[word]=tweets_count_word_year['word_frequency']
    # by month
    tweets_count_word_month = tweets.groupby([tweets['date'].map(lambda x: x[0:7])]).sum()
    words_month[word]=tweets_count_word_month['word_frequency']
    
    with pd.ExcelWriter(export_path, engine = 'openpyxl') as writer:
        writer.book = book
        words_year.to_excel(writer, sheet_name= 'yearly')
        words_month.to_excel(writer, sheet_name= 'monthly')

C:\Users\jmart130\AppData\Local\Temp\ipykernel_11748\3237516105.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tweets_count_word_year = tweets.groupby([tweets['date'].map(lambda x: x[0:4])]).sum()
C:\Users\jmart130\AppData\Local\Temp\ipykernel_11748\3237516105.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tweets_count_word_month = tweets.groupby([tweets['date'].map(lambda x: x[0:7])]).sum()
C:\Users\jmart130\AppData\Local\Temp\ipykernel_11748\3237516105.py:17: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be remo

## Frequency of specific words within a category

In [34]:
tweets = pd.read_csv('./../data/texts/colombian_valid_tweets_subject_preds_wwm.csv')
word = 'agua'
cat = 'government'
tweets = tweets[tweets['referred_to']==cat]
tweets['word_frequency'] = [min(text.count(word),1) for text in tweets['text'].astype(str).str.lower()] #Remove .lower() if case sentitivity is required

In [35]:
# by year
tweets_count_word_year = tweets.groupby([tweets['date'].map(lambda x: x[0:4])]).sum()
# by month
tweets_count_word_month = tweets.groupby([tweets['date'].map(lambda x: x[0:7])]).sum()

C:\Users\jmart130\AppData\Local\Temp\ipykernel_8916\765518570.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tweets_count_word_year = tweets.groupby([tweets['date'].map(lambda x: x[0:4])]).sum()
C:\Users\jmart130\AppData\Local\Temp\ipykernel_8916\765518570.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tweets_count_word_month = tweets.groupby([tweets['date'].map(lambda x: x[0:7])]).sum()


In [36]:
with pd.ExcelWriter('./../analysis/words_frequency/'+word+'_word_frequency_'+cat+'.xlsx') as writer:
    tweets_count_word_year['word_frequency'].to_excel(writer, sheet_name= word+' frequency - yearly')
    tweets_count_word_month['word_frequency'].to_excel(writer, sheet_name= word+' frequency - month')

## Number of tweets and by category

In [8]:
tweets_preds = pd.read_csv('./../data/texts/colombian_valid_tweets_subject_predictions_bert_wwm.csv')
tweets_preds['number of tweets'] = 1

In [9]:
# by year
tweets_count_total_year = tweets_preds.groupby(tweets_preds['date'].map(lambda x: x[0:4])).count()
tweets_count_categories_year = tweets_preds.groupby([tweets_preds['referred_to'],tweets_preds['date'].map(lambda x: x[0:4])]).count()
# by month
tweets_count_total_month = tweets_preds.groupby(tweets_preds['date'].map(lambda x: x[0:7])).count()
tweets_count_categorie_month = tweets_preds.groupby([tweets_preds['referred_to'],tweets_preds['date'].map(lambda x: x[0:7])]).count()

In [10]:
with pd.ExcelWriter('./../analysis/tweets_counts_by_subject.xlsx') as writer:
    tweets_count_total_year['number of tweets'].to_excel(writer, sheet_name= 'total - year')
    tweets_count_categories_year['number of tweets'].to_excel(writer, sheet_name= 'by categories - year ')
    tweets_count_total_month['number of tweets'].to_excel(writer, sheet_name= 'total - month')
    tweets_count_categorie_month['number of tweets'].to_excel(writer, sheet_name= 'by categories - month')

## Number of negative tweets and by category (Frustration)

In [3]:
tweets_preds = pd.read_csv('./../data/texts/colombian_valid_tweets_tone_preds_wwm.csv')

In [31]:
tweets_preds = tweets_preds[tweets_preds['tone']=='negative']
tweets_preds['number of tweets'] = 1
tweets_preds['bimonth'] = tweets_preds['date'].map(lambda x: x[0:4])+'-'+tweets_preds['date'].map(lambda x: str(math.ceil(int(x[5:7])/2)).zfill(2))

In [33]:
# by year
tweets_count_total_year = tweets_preds.groupby(tweets_preds['date'].map(lambda x: x[0:4])).count()
tweets_count_categories_year = tweets_preds.groupby([tweets_preds['referred_to'],tweets_preds['date'].map(lambda x: x[0:4])]).count()
# by month
tweets_count_total_month = tweets_preds.groupby(tweets_preds['date'].map(lambda x: x[0:7])).count()
tweets_count_categorie_month = tweets_preds.groupby([tweets_preds['referred_to'],tweets_preds['date'].map(lambda x: x[0:7])]).count()

In [34]:
# by bimonth
tweets_count_total_bimonth = tweets_preds.groupby(tweets_preds['bimonth'].map(lambda x: x[0:7])).count()
tweets_count_categorie_bimonth = tweets_preds.groupby([tweets_preds['referred_to'],tweets_preds['bimonth'].map(lambda x: x[0:7])]).count()

In [39]:
export_path = './../analysis/negative_tweets_counts_by_subject.xlsx'
from openpyxl import load_workbook
book = load_workbook(export_path)

with pd.ExcelWriter(export_path, engine = 'openpyxl') as writer:
    writer.book = book
    # tweets_count_total_year['number of tweets'].to_excel(writer, sheet_name= 'total - year')
    # tweets_count_total_month['number of tweets'].to_excel(writer, sheet_name= 'total - month')
    # tweets_count_categories_year['number of tweets'].to_excel(writer, sheet_name= 'by categories - year ')
    # tweets_count_categorie_month['number of tweets'].to_excel(writer, sheet_name= 'by categories - month')
    # tweets_count_categorie_bimonth['number of tweets'].to_excel(writer, sheet_name= 'by categories - bimonth')

C:\Users\jmart130\AppData\Local\Temp\ipykernel_10560\2669853010.py:6: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


## Tone frequency (With and without retweets)

In [63]:
tweets_preds = pd.read_csv('./../data/texts/colombian_valid_tweets_negativeness_preds_multi.csv')

In [64]:
tweets_organized_tone = []
for idx,row in tweets_preds.iterrows():
    if row['tone'] == 'negative':
        row['tone'] = row['negativeness']
    tweets_organized_tone.append(row)

In [65]:
tweets_organized_tone =pd.DataFrame.from_dict(tweets_organized_tone)
tweets_organized_tone['number of tweets'] = 1
tweets_organized_tone['bimonth'] = tweets_organized_tone['date'].map(lambda x: x[0:4])+'-'+tweets_organized_tone['date'].map(lambda x: str(math.ceil(int(x[5:7])/2)).zfill(2))

### With retweets

In [68]:
# by year
tweets_count_total_year = tweets_organized_tone.groupby([tweets_organized_tone['tone'],tweets_organized_tone['date'].map(lambda x: x[0:4])]).count()

# by month
tweets_count_total_month = tweets_organized_tone.groupby([tweets_organized_tone['tone'],tweets_organized_tone['date'].map(lambda x: x[0:7])]).count()

# by bimonth
tweets_count_total_bimonth = tweets_organized_tone.groupby([tweets_organized_tone['tone'],tweets_organized_tone['bimonth']]).count()

In [71]:
export_path = './../analysis/tone_frequencies.xlsx'
book = load_workbook(export_path)

with pd.ExcelWriter(export_path, engine = 'openpyxl') as writer:
    writer.book = book
    tweets_count_total_year['number of tweets'].to_excel(writer, sheet_name= 'yearly')
    tweets_count_total_month['number of tweets'].to_excel(writer, sheet_name= 'monthly')
    tweets_count_total_bimonth['number of tweets'].to_excel(writer, sheet_name= 'bimonthly')

C:\Users\jmart130\AppData\Local\Temp\ipykernel_10560\2738424403.py:5: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book


### Without retweets

In [73]:
tweets_organized_tone_nort = tweets_organized_tone.drop_duplicates(subset = 'text', inplace=False)

In [76]:
# by year
tweets_count_total_year = tweets_organized_tone_nort.groupby([tweets_organized_tone_nort['tone'],tweets_organized_tone_nort['date'].map(lambda x: x[0:4])]).count()

# by month
tweets_count_total_month = tweets_organized_tone_nort.groupby([tweets_organized_tone_nort['tone'],tweets_organized_tone_nort['date'].map(lambda x: x[0:7])]).count()

# by bimonth
tweets_count_total_bimonth = tweets_organized_tone_nort.groupby([tweets_organized_tone_nort['tone'],tweets_organized_tone_nort['bimonth']]).count()

In [77]:
export_path = './../analysis/tone_frequencies.xlsx'
book = load_workbook(export_path)

with pd.ExcelWriter(export_path, engine = 'openpyxl') as writer:
    writer.book = book
    tweets_count_total_year['number of tweets'].to_excel(writer, sheet_name= 'yearly - no RT')
    tweets_count_total_month['number of tweets'].to_excel(writer, sheet_name= 'monthly - no RT')
    tweets_count_total_bimonth['number of tweets'].to_excel(writer, sheet_name= 'bimonthly - no RT')

C:\Users\jmart130\AppData\Local\Temp\ipykernel_10560\579343795.py:5: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
